1️⃣ Data Preparation for Modeling

In [12]:
## 1️⃣ Prepare data and train baseline model
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import joblib

df = pd.read_csv("../data/processed/merged_clean_data.csv")

# --- target and predictors ---
target = "tree_cover_loss_ha"
features = [
    "year",
    "hard_commodities", "logging", "wildfire",
    "permanent_agriculture", "shifting_cultivation",
    "gfw_forest_carbon_net_flux__mg_co2e_yr-1",
    "carbon_gross_emissions_MgCO2e",
    "primary_forest_loss_ha",
    "umd_tree_cover_extent_2000__ha",
    "gfw_aboveground_carbon_stocks_2000__mg_c"
]

df = df.dropna(subset=[target, "country"] + features)

# encode country
df_enc = pd.get_dummies(df, columns=["country"], drop_first=True)
X = df_enc[features + [c for c in df_enc.columns if c.startswith("country_")]]
y = df_enc[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

print(f"✅ R²={r2_score(y_test, rf.predict(X_test)):.3f}")
joblib.dump({"model": rf, "features": X.columns.tolist()}, "../models/tree_cover_loss_model.pkl")


✅ R²=0.972


['../models/tree_cover_loss_model.pkl']

2️⃣ Linear extrapolation helper

In [13]:
## 2️⃣ Helper for future extrapolation
import numpy as np


def linear_extrapolate(years, values, target_year):
    """Return linear extrapolation of `values` to `target_year`."""
    s = pd.DataFrame({"year": years, "val": values}).dropna().drop_duplicates().sort_values("year")
    if len(s) >= 3:
        x, y = s["year"].values, s["val"].values
        A = np.vstack([x, np.ones_like(x)]).T
        a, b = np.linalg.lstsq(A, y, rcond=None)[0]
        return float(a * target_year + b)
    return float(s["val"].iloc[-1]) if len(s) else np.nan


3️⃣ Predict for any (country, year)

In [14]:
## 3️⃣ Predict Tree-Cover-Loss for any country and future year
bundle = joblib.load("../models/tree_cover_loss_model.pkl")
model, feature_cols = bundle["model"], bundle["features"]


def predict_tree_loss_future(country, year):
    dff = df[df["country"] == country].copy()
    if "threshold" in dff.columns:
        filt = dff[dff["threshold"] == 30]
        if not filt.empty:
            dff = filt

    if dff.empty:
        print(f"⚠️ No data for {country}")
        return None

    latest = dff.sort_values("year").tail(1).copy()

    # --- extrapolate all numeric predictors to target year ---
    for col in features:
        if col != "year":
            latest[col] = linear_extrapolate(dff["year"], dff[col], year)

    latest["year"] = year

    # encode country and align features
    enc = pd.get_dummies(latest, columns=["country"], drop_first=True)
    for c in feature_cols:
        if c not in enc.columns:
            enc[c] = 0
    X_future = enc[feature_cols].fillna(0)

    pred = model.predict(X_future)[0]
    print(f"🌲 Predicted Tree-Cover-Loss in {country} ({year}): {pred:,.0f} ha")
    return pred


# Example
predict_tree_loss_future("Brazil", 2035)
predict_tree_loss_future("Indonesia", 2030)


🌲 Predicted Tree-Cover-Loss in Brazil (2035): 2,914,564 ha
🌲 Predicted Tree-Cover-Loss in Indonesia (2030): 1,465,318 ha


np.float64(1465318.4233333333)

In [15]:
## 4️⃣ Forecast 2025–2035 for all countries
future_years = range(df["year"].max() + 1, df["year"].max() + 11)
rows = []

for country in sorted(df["country"].unique()):
    for year in future_years:
        val = predict_tree_loss_future(country, year)
        rows.append({"country": country, "year": year, "pred_tree_loss_ha": val})

pred_df = pd.DataFrame(rows)
pred_df.to_csv("../data/predicted_tree_cover_loss_2025_2035.csv", index=False)
print("✅ Saved forecast table.")

🌲 Predicted Tree-Cover-Loss in Angola (2025): 281,359 ha
🌲 Predicted Tree-Cover-Loss in Angola (2026): 286,679 ha
🌲 Predicted Tree-Cover-Loss in Angola (2027): 287,362 ha
🌲 Predicted Tree-Cover-Loss in Angola (2028): 288,107 ha
🌲 Predicted Tree-Cover-Loss in Angola (2029): 288,335 ha
🌲 Predicted Tree-Cover-Loss in Angola (2030): 287,865 ha
🌲 Predicted Tree-Cover-Loss in Angola (2031): 297,952 ha
🌲 Predicted Tree-Cover-Loss in Angola (2032): 307,342 ha
🌲 Predicted Tree-Cover-Loss in Angola (2033): 319,990 ha
🌲 Predicted Tree-Cover-Loss in Angola (2034): 320,005 ha
🌲 Predicted Tree-Cover-Loss in Argentina (2025): 188,634 ha
🌲 Predicted Tree-Cover-Loss in Argentina (2026): 165,124 ha
🌲 Predicted Tree-Cover-Loss in Argentina (2027): 144,849 ha
🌲 Predicted Tree-Cover-Loss in Argentina (2028): 125,366 ha
🌲 Predicted Tree-Cover-Loss in Argentina (2029): 121,225 ha
🌲 Predicted Tree-Cover-Loss in Argentina (2030): 104,667 ha
🌲 Predicted Tree-Cover-Loss in Argentina (2031): 101,579 ha
🌲 Predicte